# PRÁCTICA 1: Extracción de datos de una red social

Esta práctica consiste en descargar comentarios de un subreddit de Reddit, almacenarlos en disco para su posterior análisis y realizar un simple procesamiento del corpus.
Se vectoriza el corpus para obtener los términos más generales y los más repetidos.
Por último se usa la Whoosh para indexar los documentos descargados y realizar búsquedas.

En la siguiente celda de bash se crea el archivo `paraw.ini` en donde se almacenan las credenciales de acceso a la API, de esta forma se evita incluir las credenciales en el código.
Entre corchetes se pone el identificador con el cual luego se hará referencia desde el código para cargar las credenciales.

Dado que en esta práctica solo se hará uso de características de lectura de la API solo serán necesarias las credenciales de `client_id` y de `client_secret`.
[[1]](https://praw.readthedocs.io/en/stable/getting_started/authentication.html#application-only-flows)
Las otras credenciales son necesarias en caso de necesitar tener un contexto de usuario para poder hacer publicaciones de comentarios, upvotes, etc.

In [ ]:
%%bash

CLIENT_ID=""
CLIENT_SECRET=""

echo "

[tgine]
client_id=$CLIENT_ID
client_secret=$CLIENT_SECRET
" >> praw.ini

En la siguiente celda se encuentran los imports necesarios para la ejecución del script.

In [ ]:
import os

import praw
import json
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from pprint import pprint

from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID, BOOLEAN
from whoosh.qparser import QueryParser

En la siguiente celda se definen constantes globales.

* `subreddit_display_name` El subreddit elegido es [science](https://www.reddit.com/r/science/).
* `collections_dir` La carpeta donde se guardaran las colecciones.
* `collection_names` Los tres modos de obtener comentarios, se corresponden con los nombres de las colecciones
* `reddit_client_user_agent` El _User-Agent_ que se usará con el cliente.
* `min_document_frequency` Mínima frecuencia por término en los documentos.
* `most_central_terms` Los términos más centrales que se mostrarán.
* `most_repeated` Los términos más repetidos que se mostraran.

In [ ]:
subreddit_display_name = "science"
collections_dir = "collections"
collection_names = ["new", "hot", "rising"]
reddit_client_user_agent = "python:com.example.gonzalocl1024.tgine:v1.0 (by /u/gonzalocl1024)"

min_document_frequency = 10
most_central_terms = 50
most_repeated = 100

En la siguiente celda se definen los atributos que se copiarán de cada uno de los objetos que devuelva la librería para cada uno de los comentarios.
La función `copy_attributes` copia los atributos que son primitivos y serializables, en caso de que un atributo no exista saltará una excepción que será capturada.

Cada una de las funciones copia un tipo de objeto respectivamente.

In [ ]:
subreddit_attributes = [
    "display_name",
    "title",
    "active_user_count",
    "subscribers",
    "id",
    "description",
    "created_utc",
    "name"
]

submission_attributes = [
    "title",
    "name",
    "upvote_ratio",
    "ups",
    "score",
    "id",
    "created_utc",
    "selftext",
    "downs",
    "url"
]

comment_attributes = [
    "ups",
    "id",
    "score",
    "body",
    "downs"
]

def copy_attributes(dest, src, attributes):
    for attribute in attributes:
        try:
            dest[attribute] = src[attribute]
        except KeyError:
            i = None
            n = None
            try:
                i = src["id"]
            except KeyError:
                pass
            try:
                n = src["name"]
            except KeyError:
                pass
            print("\n{}, {}, {}".format(attribute, i, n))

def copy_subreddit(subreddit):
    dest = {}
    copy_attributes(dest, subreddit, subreddit_attributes)
    return dest

def copy_submission(submission):
    dest = {}
    copy_attributes(dest, submission, submission_attributes)
    return dest

def copy_comment(comment):
    dest = {}
    copy_attributes(dest, comment, comment_attributes)
    return dest

def copy_author(dest, src):
    if src.author:
        dest["author"] = src.author.name
    else:
        dest["author"] = None

A continuación se definen las funciones que se encargaran de descargarse los comentarios del subreddit.
La primera función `retrieve_submissions` descarga todos los submissions que pueda y devuelve una lista de objetos submission.

La función siguiente función `retrieve_comments`, se encarga de descargar todos los comentarios de todos los submissions que se han obtenido en la función anterior.
Para cada submision se copia el contenido del post inicial y además con la línea `submission.comments.replace_more(limit=None)`, el wrapper se encarga de [descargar todos los comentarios](https://praw.readthedocs.io/en/stable/tutorials/comments.html#the-replace-more-method) de ese submision.
Se itera sobre todos los comentarios se copian de los objetos creados por el wrapper a diccionarios de Python y se almacenan en la colección que se pasó por parámetro.
El wrapper convierte el campo autor en un objeto por lo que se copia solo el username del autor, para que posteriormente sea posible serializar los datos copiados.

Por último la función `get_collection` se encarga de llamar a las dos funciones anteriores.
Primero copia la información del subreddit, luego se descarga la colección que se le pasa por parámetro, y después se descargan todos los comentarios.

En la linea `getattr(subreddit, name)(limit=None)`, el parámetro `name` corresponde con el nombre de la colección (_new_, _hot_, _rising_), y con el parámetro `limit=None` se le indica que se descargue el máximo número de submision de esa colección posible.

La estructura del diccionario devuelto es la siguiente:

```
{'subreddit': {<información sobre el subreddit>},
 'submissions': [{'submission': {<información sobre el submission>}, 'comments': [<lista de comentarios>]},
                 {'submission': {<información sobre el submission>}, 'comments': [<lista de comentarios>]},
                 ...]}
```


In [ ]:
def retrieve_submissions(submissions):

    submission_list = []
    i = 0

    for submission in submissions:
        submission_list.append(submission)

        i += 1
        print("\rRetrieved submissions: {}".format(i), end="")

    print()
    return submission_list

def retrieve_comments(collection, submissions):

    i = 0
    total_submissions = len(submissions)
    total_comments = 0

    collection["submissions"] = []

    for submission in submissions:

        _ = submission.title
        submission_copy = copy_submission(vars(submission))
        copy_author(submission_copy, submission)

        print(" Next comment batch: {:<35}".format(submission.num_comments), end="")

        comments = []
        submission.comments.replace_more(limit=None)

        for comment in submission.comments.list():
            _ = comment.body
            comment_copy = copy_comment(vars(comment))
            copy_author(comment_copy, comment)
            comments.append(comment_copy)

        collection["submissions"].append({
            "submission": submission_copy,
            "comments": comments
        })

        i += 1
        total_comments += len(comments)
        print("\rRetrieved comments: {} (Submissions: {}/{})".format(total_comments, i, total_submissions), end="")

    print()

def get_collection(subreddit, name):

    _ = subreddit.title

    submissions = getattr(subreddit, name)(limit=None)

    collection = {}

    # add subreddit info and date
    collection["collection_name"] = name
    collection["subreddit"] = copy_subreddit(vars(subreddit))
    collection["date"] = datetime.datetime.now().isoformat()

    # retrieve submissions list
    submission_list = retrieve_submissions(submissions)

    # retrieve comments
    retrieve_comments(collection, submission_list)

    return collection

La función `get_collections` se encarga de cargar las colecciones.
En primer lugar se comprueba si la colección ya ha sido descargada y está almacenada en disco, en tal caso se carga de disco el JSON de la colección.

Si por el contrario la colección no se encontrase en disco se descargaría haciendo uso de las funciones descritas anteriormente.
En este caso se guardará la colección en disco en formato JSON para ser usada en futuras ejecuciones.

Toma por parámetros el nombre del subreddit a descargar y los nombres de los modos de descargar las colecciones (_new_, _hot_, _rising_) que se corresponden con los nombres de las colecciones.
Devuelve un diccionario con una clave por colección, cada cual contiene la colección correspondiente.

A la hora de instanciar el objeto cliente de la API de reddit (`praw.Reddit("tgine", user_agent=reddit_client_user_agent)`), en vez de pasarle por parámetros las credenciales, se le pasa el identificador para que lea las credenciales que se almacenaron en el archivo `praw.ini` anteriormente.

In [ ]:
def get_collection_path(collection_dir, display_name, collection_name):
    return os.path.join(collection_dir, "{}_{}.json".format(display_name, collection_name))

def get_collections(display_name, names):

    reddit_client = None
    subreddit = None

    os.makedirs(collections_dir, exist_ok=True)

    collections = {}

    for collection_name in names:

        collection_filename = get_collection_path(collections_dir, display_name, collection_name)

        if os.path.isfile(collection_filename):
            # read collection
            print("Using saved collection: {}".format(collection_filename))
            with open(collection_filename) as collection_file:
                collection = json.load(collection_file)

            total_comments = 0
            for submission in collection["submissions"]:
                total_comments += len(submission["comments"])
            print("  Submissions: {}, Comments: {}".format(len(collection["submissions"]), total_comments))

        else:
            # retrieve collection

            if not reddit_client:
                reddit_client = praw.Reddit("tgine", user_agent=reddit_client_user_agent)
                subreddit = reddit_client.subreddit(display_name)

            collection = get_collection(subreddit, collection_name)

            print("Saving collection: {}".format(collection_filename))
            with open(collection_filename, "w") as collection_file:
                json.dump(collection, collection_file)

        collections[collection_name] = collection

    return collections

La función `extract_corpus` se encarga de tomar las colecciones anteriores, que consisten en diccionarios con la información del subrredit, submissions y comentarios y convertir eso en un diccionario con una estructura más simple que contenga simplemente el texto de las submissions y los comentarios.

Para cada colección se crea un corpus distinto, en cuanto al post inicial (submission), el texto del título y el contenido del post se concatenan.
El contenido de los comentarios se incluye todo en la misma lista.
Para cada colección se crea una lista donde cada elemento de la lista será un documento.

Por último se descartan todos los comentarios que sean del usuario `AutoModerator`, que es un bot.
Se descartan los mensajes del bot porque en todos los submisions el primer comentario es de este bot y en todas las ocasiones es el mismo mensaje ([ejemplo](https://www.reddit.com/r/science/comments/nyhcip/study_employees_who_experienced_higher_levels_of/h1k2vhx/)), lo que añade ruido en las colecciones, sobre todo en las colecciones en las que por lo general no hay muchos comentarios por submission.
Como por ejemplo en la colección _new_ donde al estar menos tiempo publicadas las submissions tienen menos comentarios.

La estructura del diccionario devuelto es la siguiente:

```
{'new': [<lista de documentos>],
 'hot': [<lista de documentos>],
 'rising': [<lista de documentos>]}
```

In [ ]:
def extract_corpus(collections):

    corpus = {}

    for collection in collections:
        corpus[collection] = []

        for submission in collections[collection]["submissions"]:

            corpus[collection].append("{} {}".format(submission["submission"]["title"],
                                                     submission["submission"]["selftext"]))

            for comment in submission["comments"]:

                # AutoModerator is a bot discard its messages
                if not comment["author"] == "AutoModerator":
                    corpus[collection].append(comment["body"])

    return corpus

A continuación se cargan las colecciones y seguidamente se extrae el corpus.

In [ ]:
subreddit_collections = get_collections(subreddit_display_name, collection_names)
collections_corpus = extract_corpus(subreddit_collections)

En la siguiente función se vectoriza el corpus extraído anteriormente y se muestran los términos más centrales y los más repetidos.

Para la vectorización al objeto vectorizador se le pasa el parámetro `min_df=min_doc_freq` con el que se le indica que descarte los términos menos frecuentes que `min_doc_freq`.
Con el parámetro `max_features=most_rep` se le indica que se quede solo con los `most_rep` términos, los más repetidos del corpus.
Como se indica en la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

Con `vectorizer.get_feature_names()` se obtienen los términos más repetidos de la colección.

Con `np.sum(tfidf.toarray(), axis=0)` se obtienen las puntuaciones de los términos más centrales, calculadas como la suma de todas sus puntuaciones a través de todos los documentos.
La puntuación resultante en cada posición de la lista se corresponde con el término devuelto por el método `get_feature_names`.
Para obtener los términos más centrales se tienen que obtener los índices correspondientes con las puntuaciones más altas, para ello se usa el método `np.argsort`.

In [ ]:
def print_central_repeated(corpus, min_doc_freq, most_central, most_rep):

    vectorizer = TfidfVectorizer(stop_words="english", min_df=min_doc_freq, max_features=most_rep)
    tfidf = vectorizer.fit_transform(corpus)

    most_repeated_terms = vectorizer.get_feature_names()

    central_terms_score = np.sum(tfidf.toarray(), axis=0)
    central_terms_indexes = np.argsort(central_terms_score)[-most_central:]
    central_terms = [most_repeated_terms[i] for i in central_terms_indexes]
    central_terms.reverse()

    print("{} most central terms".format(most_central))
    pprint(central_terms, width=100, compact=True)
    print()

    print("{} most repeated terms".format(most_rep))
    pprint(most_repeated_terms, width=100, compact=True)
    print("\n")

A continuación se muestran los términos más centrales y más repetidos para cada función.

In [ ]:
for collection_corpus in collections_corpus:

    print("Collection: {}".format(collection_corpus))
    print_central_repeated(collections_corpus[collection_corpus], min_document_frequency, most_central_terms, most_repeated)

A continuación se realiza el apartado opcional _Utilizad Whoosh para indexar los documentos_.

Las siguientes funciones servirán para hacer uso del índice de documentos.
La función `get_index` se usa para cargar el índice, en caso de que el índice ya haya sido creado y exista en disco, ese índice es cargado, en caso contrario se crea.

Para crear el índice primero se define el esquema, el esquema se compone de los campos:

*  _collection_: indica a la colección que pertenece (_new_, _hot_, _rising_).
*  *submission_id*: es el identificador del submission, puede servir para filtrar por submission.
*  _author_: autor del contenido.
*  _content_: el texto del submission o comentario.
*  *is_submission*: indica si el documento indexado es submission o comentario.

Añadiendo `stored=True` se indica que se almacene el dato para que a la hora de hacer consultas pueda ser consultado y mostrado, en caso contrario solo se podrán hacer búsquedas con ese dato, pero no mostrarlo.

Seguidamente se crea el objeto indexador, y se llama a la función `add_collections` que añadirá todos los documentos de todas las colecciones al índice.

Por último la función `print_results` se puede usar para mostrar los resultados obtenidos en una consulta de una forma legible.

In [ ]:
def add_collections(writer, collections):

    total_submissions = 0
    submissions = 0
    for collection in collections:
        total_submissions += len(collections[collection]["submissions"])

    for collection in collections:
        for submission in collections[collection]["submissions"]:

            writer.add_document(collection=collection,
                                submission_id=submission["submission"]["id"],
                                author=submission["submission"]["author"],
                                content="{}\n{}".format(submission["submission"]["title"],
                                                        submission["submission"]["selftext"]),
                                is_submission=True)

            total_comments = len(submission["comments"])
            comments = 0
            submissions += 1

            for comment in submission["comments"]:
                writer.add_document(collection=collection,
                                    submission_id=submission["submission"]["id"],
                                    author=comment["author"],
                                    content=comment["body"],
                                    is_submission=False)

                comments += 1
                print("\rIndexed: Submissions {}/{}; Comments {}/{}               ".format(submissions,
                                                                                           total_submissions,
                                                                                           comments,
                                                                                           total_comments), end="")
    print()

def get_index(index_path, collections):

    if os.path.exists(index_path):
        print("Using saved index: {}".format(index_path))
        return open_dir(index_path)

    print("Creating index: {}".format(index_path))
    os.mkdir(index_path)

    schema = Schema(collection=TEXT(stored=True),
                    submission_id=ID(stored=True),
                    author=TEXT(stored=True),
                    content=TEXT(stored=True),
                    is_submission=BOOLEAN(stored=True))

    index = create_in(index_path, schema)

    writer = index.writer()
    add_collections(writer, collections)
    writer.commit()

    return index

def print_results(results):

    print("Search runtime: {}".format(results.runtime))
    print("Total results: {} (showing {})\n".format(results.estimated_length(),
                                                    results.scored_length()))

    for result in results:
        print("{:<20}{}".format(result["author"], result["content"].strip()))

En la siguiente celda se carga el índice haciendo uso de las funciones antes descritas.
Primero se carga el índice y luego se obtiene el objeto `index_searcher` que es necesario para poder hacer consultas al índice.

También se instancia un objeto `QueryParser` que servirá para poder hacer consultas sin necesidad de crear un objeto consulta con código Python.
Con este objeto se puede hacer una consulta a partir de una consulta textual usando la sintaxis adecuada.
Al objeto `QueryParser` hay que pasarle por parámetros el campo por defecto que se usará para buscar (en este caso 'content') y el esquema del índice.

In [ ]:
collections_index = get_index("index", subreddit_collections)
index_searcher = collections_index.searcher()
query_parser = QueryParser("content", collections_index.schema)

A continuación se muestra un ejemplo de consulta al índice.
Primero se obtiene el objeto de consulta a partir de la cadena de consulta usando el objeto `QueryParser`.
Seguidamente se hace la búsqueda en el índice con el método `search` y se muestran los resultados.

La sintaxis que se ha de usar para la cadena de consulta son pares de campos seguidos del valor que se quiere buscar en ese campo, separados por `:`.
En caso de que no se indique el campo en el que se quiere buscar, se buscará en el campo por defecto especificado en la instanciación del objeto `QueryParser`.

```
<consulta en campo por defecto> <campo>:<consulta en el campo> <campo>:<consulta en el campo> ...
```

Especificar el campo donde buscar es una opción del lenguaje de consulta, el lenguaje proporciona más características como operaciones lógicas (AND y OR).

In [ ]:
query_str = "scientists is_submission:true"

query = query_parser.parse(query_str)
search_results = index_searcher.search(query)
print_results(search_results)

Para finalizar el índice se cierra para liberar los recursos.

In [ ]:
index_searcher.close()
collections_index.close()